# Workflow
### Data preprocess
* file directory에서 데이터 불러오기
* 데이터 형식에 맞춰 DataFrame으로 만들기 (dictionary 형태 이용)
* 필요하지 않은 column 제거 및 결측치 처리
* graph구조로 만들기위해 node,edge가 될 attribute들 정하기
* long tailed 제거 (optional)
* dgl library를 이용해 grpah structure로 변환
* split train/test/validation 
* preprocessed data 저장

### Model
* TODO

### Metrics
* TODO

### Evaluation
* Metrics랑 같이넣을지 고민

### Test
* 결과값 및 분석 그래프 시각화

In [ ]:
# 데이터 전처리

import os
import re
import argparse
import pickle
import pandas as pd
import numpy as np
import scipy.sparse as ssp
import dgl
import torch
import torchtext
from builder import PandasGraphBuilder
from data_utils import *
from copy import deepcopy

def remove_infrequent_users(data, min_counts=10):
    '''Remove infrequent users'''
    df = deepcopy(data)
    counts = df['UserId'].value_counts()
    df = df[df['UserId'].isin(counts[counts >= min_counts].index)]
    
    return df    

def remove_infrequent_items(data, min_counts=10):
    '''Remove infrequent items'''
    df = deepcopy(data)
    counts = df['ProductId'].value_counts()
    df = df[df['ProductId'].isin(counts[counts >= min_counts].index)]
    
    return df

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('directory', type=str)
#     parser.add_argument('output_path', type=str)
#     args = parser.parse_args()
#     directory = args.directory
#     output_path = args.output_path

#     # Read amazon beauty dataset
#     #amazon = pd.read_csv('./amazon_beauty/ratings_Beauty.csv')
#     amazon = pd.read_csv(os.path.join(directory, 'ratings_Beauty.csv'))

#     # Separate user, item columns
#     # user_col = amazon.drop(['ProductId','Rating','Timestamp'], axis=1).astype('category')
#     # item_col = amazon.drop(['UserId','Rating','Timestamp'], axis=1).astype('category')
    

#     # Filter out user,item with fewer than 10 interactions
#     filtered_data = remove_infrequent_users(amazon, 10)
#     filtered_data = remove_infrequent_items(filtered_data, 10)
    
#     # Get distinct users and items
#     distinct_users = filtered_data['UserId'].unique()
#     distinct_items = filtered_data['ProductId'].unique()
#     users = pd.DataFrame(distinct_users, columns=['UserId'])
#     items = pd.DataFrame(distinct_items, columns=['ProductId'])

#     # Build graph
#     graph_builder = PandasGraphBuilder()
#     graph_builder.add_entities(users, 'UserId', 'user')
#     graph_builder.add_entities(items, 'ProductId', 'item')
#     graph_builder.add_binary_relations(filtered_data, 'UserId','ProductId','purchased')
#     graph_builder.add_binary_relations(filtered_data, 'ProductId', 'UserId', 'purchased-by')

#     g = graph_builder.build()

#     # Add user-item edge features
#     g.edges['purchased'].data['rating'] = torch.LongTensor(filtered_data['Rating'].values)
#     g.edges['purchased'].data['timestamp'] = torch.LongTensor(filtered_data['Timestamp'].values)
#     g.edges['purchased-by'].data['rating'] = torch.LongTensor(filtered_data['Rating'].values)
#     g.edges['purchased-by'].data['timestamp'] = torch.LongTensor(filtered_data['Timestamp'].values)

#     # Train, Test, Validation split by time
#     train_indices, val_indices, test_indices = train_test_split_by_time(filtered_data, 'Timestamp', 'UserId')

#     # Build Train Graph
#     train_g = build_train_graph(g, train_indices, 'user','item','purchased','purchased-by')
#     assert train_g.out_degrees(etype='purchased').min() > 0

#     val_matrix, test_matrix = build_val_test_matrix(g, val_indices, test_indices, 'user','item','purchased')

#     # Item text set
#     item_text_dataset = {'productid' : items['ProductId'].values}

#     dataset = {
#         'train-graph': train_g,
#         'val-matrix': val_matrix,
#         'test-matrix': test_matrix,
#         'item-texts': item_text_dataset,
#         'item-images': None,
#         'user-type': 'user',
#         'item-type': 'item',
#         'user-to-item-type': 'purchased',
#         'item-to-user-type': 'purchased-by',
#         'timestamp-edge-column': 'timestamp'}

#     with open(output_path, 'wb') as f:
#         pickle.dump(dataset, f)

#     with open('processed_beauty_filtered.pkl', 'rb') as fr:
#         data = pickle.load(fr)
#     print(data)

In [ ]:
# movielens 